In [1]:
import os 
import glob
from scipy.io import loadmat, savemat
import pickle as pkl

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from scipy.signal import medfilt

from scipy.signal import resample
from scipy.interpolate import interp1d

from scipy.linalg import svd
from scipy import signal

In [2]:
train_pkls = glob.glob('../../../datasets/Data_27012021_noisy/train/*.pkl')
print ('Train:', len(train_pkls))
val_pkls = glob.glob('../../../datasets/Data_27012021_noisy/val/*.pkl')
print ('Val:', len(val_pkls))
benchmark_pkls = glob.glob('../../../datasets/Data_27012021_noisy/benchmark/*.pkl')
print ('Benchmark:', len(benchmark_pkls))
raw_mats = glob.glob('../../../datasets/Data_27012021_noisy/raw/*.mat')
print ('Raw:', len(raw_mats))

Train: 100
Val: 100
Benchmark: 4
Raw: 10


In [3]:
fin = open(train_pkls[0], 'rb')
train_sample = pkl.load(fin)
fin.close()

fin = open(val_pkls[0], 'rb')
val_sample = pkl.load(fin)
fin.close()

fin = open(benchmark_pkls[0], 'rb')
benchmark_sample = pkl.load(fin)
fin.close()

raw_sample = loadmat(raw_mats[0])

In [4]:
print ('Train Sample Keys')
print (train_sample.keys())
print (train_sample['speed'].min(), train_sample['torque'].min())
print (train_sample['speed'].max(), train_sample['torque'].max())

print ('\n\nVal Sample Keys')
print (val_sample.keys())
print (val_sample['speed'].min(), val_sample['torque'].min())
print (val_sample['speed'].max(), val_sample['torque'].max())

print ('\n\nBenchmark Sample Keys')
print (benchmark_sample.keys())
print (benchmark_sample['speed'].min(), benchmark_sample['torque'].min())
print (benchmark_sample['speed'].max(), benchmark_sample['torque'].max())


print ('\n\nRaw Sample Keys')
print (raw_sample.keys())
print (raw_sample['Speed_Hz'].min(), raw_sample['Torque_PercTnom'].min())
print (raw_sample['Speed_Hz'].max(), raw_sample['Torque_PercTnom'].max())

Train Sample Keys
dict_keys(['voltage_d', 'voltage_q', 'current_d', 'current_q', 'torque', 'speed', 'statorPuls', 'time', 'reference_torque_interp', 'reference_speed_interp', 'reference_torque', 'reference_speed', 'torque_time', 'speed_time', 'noisy_current_d', 'noisy_current_q', 'noisy_voltage_d', 'noisy_voltage_q'])
-33.94910204460247 -90.64906259505035
58.345056599825334 226.155861206348


Val Sample Keys
dict_keys(['voltage_d', 'voltage_q', 'current_d', 'current_q', 'torque', 'speed', 'statorPuls', 'time', 'reference_torque_interp', 'reference_speed_interp', 'reference_torque', 'reference_speed', 'torque_time', 'speed_time', 'noisy_current_d', 'noisy_current_q', 'noisy_voltage_d', 'noisy_voltage_q'])
-4.172530142046642 -6.6455594472196085
0.46461061165822387 108.36099820460994


Benchmark Sample Keys
dict_keys(['voltage_d', 'voltage_q', 'current_d', 'current_q', 'torque', 'speed', 'statorPuls', 'time', 'reference_torque_interp', 'reference_speed_interp', 'reference_torque', 'refere

In [5]:
def make_mat(pkl_path):
    fin = open(pkl_path, 'rb')
    data = pkl.load(fin)
    fin.close()
    savemat(pkl_path.replace('pkl', 'mat'), data)
    

for pkl_path in train_pkls + val_pkls + benchmark_pkls:
    make_mat(pkl_path)

In [21]:
def process_raw_data(mat_path):
    print (mat_path)
    data = loadmat(mat_path)
    
    time = data['Time_VSpdTq'][:, 0]
    speed = data['Speed_Hz'][:, 0]
    torque = data['Torque_PercTnom'][:, 0]
    voltage_d = data['VdRef_V'][: ,0]
    voltage_q = data['VqRef_V'][:, 0]

    current_time = data['Time_I'][:, 0]
    current_d = data['IdMeas_A'][:, 0]
    current_q = data['IqMeas_A'][:, 0]
    
    speed_locs = np.where(speed > 240)
    for loc in speed_locs:
        speed[loc] = (speed[loc-1] + speed[loc+1]) / 2
    torque_locs = np.where(torque > 700)
    for loc in torque_locs:
        torque[loc] = (torque[loc-1] + torque[loc+1]) / 2
#     voltage_d_locs = np.where(voltage_d > 300)
#     for loc in voltage_d_locs:
#         voltage_d[loc] = (voltage_d[loc-1] + voltage_d[loc+1]) / 2
#     voltage_q_locs = np.where(voltage_q > 300)
#     for loc in voltage_q_locs:
#         voltage_q[loc] = (voltage_q[loc-1] + voltage_q[loc+1]) / 2
    current_d_locs = np.where(current_d > 22)
    for loc in current_d_locs:
        current_d[loc] = (current_d[loc-1] + current_d[loc+1]) / 2
    current_q_locs = np.where(current_q > 22)
    for loc in current_q_locs:
        current_q[loc] = (current_q[loc-1] + current_q[loc+1]) / 2
        
    
    print (speed.mean(), speed.max())
    print (torque.mean(), torque.max())
    print (voltage_d.mean(), voltage_d.max())
    print (voltage_q.mean(), voltage_q.max())
    print (current_d.mean(), current_d.max())
    print (current_q.mean(), current_q.max())
    
    f = interp1d(np.arange(current_time.shape[0]), current_d, 'linear')
    
    current_d_down = ResampleLinear1D(current_d, time.shape[0])
    current_q_down = ResampleLinear1D(current_q, time.shape[0])
    
    print ('Time', time.shape, time[1] - time[0])
    print ('Voltage d', voltage_d.shape, voltage_d.min(), voltage_d.max())
    print ('Votlage q', voltage_q.shape, voltage_q.min(), voltage_q.max())
    print ('Speed', speed.shape, speed.min(), speed.max())
    print ('Torque', torque.shape, torque.min(), torque.max())
    print ('Current d', current_d_down.shape, current_d_down.min(), current_d_down.max())
    print ('Current q', current_q_down.shape, current_q_down.min(), current_q_down.max())
    
    w = 51
    w_t = "hamming"
    denoised_speed = smooth(speed, window_len=w, window=w_t)
    denoised_torque = smooth(torque, window_len=w, window=w_t)  
    
    data = {'time': time, 
            'noisy_current_d': current_d_down, 
            'noisy_current_q': current_q_down, 
            'noisy_voltage_d': voltage_d, 
            'noisy_voltage_q': voltage_q,
            'noisy_speed': speed,
            'noisy_torque': torque,
            'speed': denoised_speed[w//2:-1*w//2+1],
            'torque': denoised_torque[w//2:-1*w//2+1]
            }
    
    savemat(mat_path.replace('raw', 'real'), data)
    
for mat_path in raw_mats:
    process_raw_data(mat_path)

../../../datasets/Data_27012021_noisy/raw/OFVC4kW_LM50Hz_paliers_0to120PercTnom_Data.mat
51.443102399628586 53.76434326171875
49.299140502798416 126.9287109375
-12.079650073378232 11.045928955078125
311.8278045400705 324.0982360839844
5.97760109198569 7.0029296875
5.289508197752048 15.14990234375
Time (10229,) 0.004
Voltage d (10229,) -60.935302734375 11.045928955078125
Votlage q (10229,) 294.3613586425781 324.0982360839844
Speed (10229,) 49.75128173828125 53.76434326171875
Torque (10229,) -2.3193359375 126.9287109375
Current d (10229,) -8.2752116012719 6.915466618901541
Current q (10229,) -0.2146105894969807 14.980709572718254
../../../datasets/Data_27012021_noisy/raw/OFVC4kW_LM30Hz_paliers_0to120PercTnom_Data.mat
30.649023098879482 32.92083740234375
44.14365827676304 120.80078125
-8.167332307368802 27.404022216796875
220.93348075328663 253.24127197265625
8.558031120516318 9.585693359375
4.113199464559229 12.185791015625
Time (10238,) 0.004
Voltage d (10238,) -48.034332275390625 27.40

In [14]:
train_real_mats = glob.glob('../../../datasets/Data_27012021_noisy/train_real/*.mat')
print ('Train Real:', len(train_real_mats))

Train Real: 3


In [15]:
for train_real_mat in train_real_mats:
    data = loadmat(train_real_mat)
    
    current_d_clean_statics = []
    for item in data['current_d_clean_statics']:
        current_d_clean_statics.append([item[0][0][0], item[0][0][1], item[1][0][0]])
    
    current_q_clean_statics = []
    for item in data['current_q_clean_statics']:
        current_q_clean_statics.append([item[0][0][0], item[0][0][1], item[1][0][0]])
    
    voltage_d_clean_statics = []
    for item in data['voltage_d_clean_statics']:
        voltage_d_clean_statics.append([item[0][0][0], item[0][0][1], item[1][0][0]])
    
    voltage_q_clean_statics = []
    for item in data['voltage_q_clean_statics']:
        voltage_q_clean_statics.append([item[0][0][0], item[0][0][1], item[1][0][0]])
    
    speed_clean_statics = []
    for item in data['speed_clean_statics']:
        speed_clean_statics.append([item[0][0][0], item[0][0][1], item[1][0][0]])
    
    torque_clean_statics = []
    for item in data['torque_clean_statics']:
        torque_clean_statics.append([item[0][0][0], item[0][0][1], item[1][0][0]])
    
    data['current_d_clean_statics'] = current_d_clean_statics
    data['current_q_clean_statics'] = current_q_clean_statics
    data['voltage_d_clean_statics'] = voltage_d_clean_statics
    data['voltage_q_clean_statics'] = voltage_q_clean_statics
    data['speed_clean_statics'] = speed_clean_statics
    data['torque_clean_statics'] = torque_clean_statics
    
    savemat(train_real_mat, data)

IndexError: invalid index to scalar variable.